In [10]:
import os
import nibabel as nib
import numpy as np
from scipy.io import savemat
from scipy.ndimage import zoom

Extract all the files ending in t1.nii.gz from the MICCAI_BraTS_2018_Data_Training directory and subdirectories

In [11]:
def extract_files(src_folder, dst_folder, file_suffix):
    os.makedirs(dst_folder, exist_ok=True)
    
    for root, dirs, files in os.walk(src_folder):
        for file in files:
            if file.endswith(file_suffix):
                src_file_path = os.path.join(root, file)
                dst_file_path = os.path.join(dst_folder, file)
                
                with open(src_file_path, 'rb') as src_file:
                    with open(dst_file_path, 'wb') as dst_file:
                        dst_file.write(src_file.read())
                        
                print(f"Copied {file} to {dst_folder}")

src_folder = '/Users/elizabethnemeti/Desktop/MICCAI_BraTS_2018_Data_Training'
dst_folder = './Extracted_t1_files'
file_suffix = 't1.nii.gz'

extract_files(src_folder, dst_folder, file_suffix) # did they get extracted?

Copied Brats18_TCIA10_639_1_t1.nii.gz to ./Extracted_t1_files
Copied Brats18_TCIA13_630_1_t1.nii.gz to ./Extracted_t1_files
Copied Brats18_2013_6_1_t1.nii.gz to ./Extracted_t1_files
Copied Brats18_TCIA13_615_1_t1.nii.gz to ./Extracted_t1_files
Copied Brats18_2013_8_1_t1.nii.gz to ./Extracted_t1_files
Copied Brats18_2013_24_1_t1.nii.gz to ./Extracted_t1_files
Copied Brats18_TCIA10_490_1_t1.nii.gz to ./Extracted_t1_files
Copied Brats18_TCIA10_637_1_t1.nii.gz to ./Extracted_t1_files
Copied Brats18_TCIA13_634_1_t1.nii.gz to ./Extracted_t1_files
Copied Brats18_TCIA10_346_1_t1.nii.gz to ./Extracted_t1_files
Copied Brats18_TCIA10_202_1_t1.nii.gz to ./Extracted_t1_files
Copied Brats18_TCIA09_312_1_t1.nii.gz to ./Extracted_t1_files
Copied Brats18_TCIA13_624_1_t1.nii.gz to ./Extracted_t1_files
Copied Brats18_TCIA10_442_1_t1.nii.gz to ./Extracted_t1_files
Copied Brats18_TCIA10_152_1_t1.nii.gz to ./Extracted_t1_files
Copied Brats18_TCIA13_645_1_t1.nii.gz to ./Extracted_t1_files
Copied Brats18_TCIA

1. converting .nii.gz files to .mat 
2. correct dimensions of new .mat files
       - need to select a middle slice randomly since the NIFTIs had an extra dimension of slice count
       - added dimensionality checks to ensure it fits preprocessing in main pipeline and Cheng dataset format
       - add channel dimension to make it (256, 256, 1)

In [14]:
def resize_data(data, target_shape=(256, 256)):
    resized_data = np.array([zoom(slice, target_shape / np.array(slice.shape), order=1) for slice in data])
    middle_slice = resized_data[resized_data.shape[0] // 2] # selecting a middle slice
    return middle_slice

def process_and_convert_nii_to_mat(src_folder, dst_folder):
    os.makedirs(dst_folder, exist_ok=True)
    
    for file in os.listdir(src_folder):
        if file.endswith('.nii.gz'):
            file_path = os.path.join(src_folder, file)
            img = nib.load(file_path)
            data = img.get_fdata().astype(np.float32)
            
            # resize for (256, 256, 1) requirement
            data = resize_data(data)
            data = data[..., np.newaxis]  # add channel dimension

            mask = np.zeros((256, 256, 3), dtype=np.float32)
            
            mat_file_path = os.path.join(dst_folder, file.replace('.nii.gz', '.mat'))
            savemat(mat_file_path, {'image': data, 'mask': mask})
            
            print(f'Saved to {mat_file_path}')
            print(f'File is a .mat file: {mat_file_path.endswith(".mat")}')
            print(f'Image shape: {data.shape} (should be (256, 256, 1))')
            print(f'Mask shape: {mask.shape} (should be (256, 256, 3))')
            print(f'Data type of image: {data.dtype == np.float32}')
            print(f'Data type of mask: {mask.dtype == np.float32}')

src_folder = './Extracted_t1_files'
dst_folder = '/Users/elizabethnemeti/Desktop/benchmark_data_brats' # for local saves

process_and_convert_nii_to_mat(src_folder, dst_folder) # are the required shapes and dimensiosn met? 

Saved to /Users/elizabethnemeti/Desktop/benchmark_data_brats/Brats18_TCIA13_642_1_t1.mat
File is a .mat file: True
Image shape: (256, 256, 1) (should be (256, 256, 1))
Mask shape: (256, 256, 3) (should be (256, 256, 3))
Data type of image: True
Data type of mask: True
Saved to /Users/elizabethnemeti/Desktop/benchmark_data_brats/Brats18_TCIA10_299_1_t1.mat
File is a .mat file: True
Image shape: (256, 256, 1) (should be (256, 256, 1))
Mask shape: (256, 256, 3) (should be (256, 256, 3))
Data type of image: True
Data type of mask: True
Saved to /Users/elizabethnemeti/Desktop/benchmark_data_brats/Brats18_2013_0_1_t1.mat
File is a .mat file: True
Image shape: (256, 256, 1) (should be (256, 256, 1))
Mask shape: (256, 256, 3) (should be (256, 256, 3))
Data type of image: True
Data type of mask: True
Saved to /Users/elizabethnemeti/Desktop/benchmark_data_brats/Brats18_TCIA13_615_1_t1.mat
File is a .mat file: True
Image shape: (256, 256, 1) (should be (256, 256, 1))
Mask shape: (256, 256, 3) (sh